# Projeto 2 - Ciência dos Dados 

### Participantes:

- Bernardo Cunha Capoferri

- Francisco Pinheiro Janela

- Henrique Martinelli Frezzatti

- Lívia Sayuri Makuta



## Objetivo:

No Projeto 2 a equipe escolheu analisar músicas que vão aparecer no Top 100 da Billboard e classificá-las no gênero RAP, por meio de características atribuidas à música analisada. Para isso vamos usar um classificador de modelo `Random Forest`, em cima de uma lista de Treinamento que possui músicas e suas características que já foram da Top 100 semanal.

Carregando algumas bibliotecas que serão usadas no Projeto:

In [1]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
import os
import re

Para começar a análise, vamos carregar o DataSet escolhido:

In [2]:
filename = 'Hot_100_Audio_Features.xlsx'
if filename in os.listdir():
    print("DataSet ok!")
else:
    print("Problema no Loading...")
    
DataFrame = pd.read_excel(filename)
DataFrame.head()

DataSet ok!


,SongID,Performer,Song,spotify_genre,spotify_track_id,spotify_track_preview_url,spotify_track_album,spotify_track_explicit,spotify_track_duration_ms,spotify_track_popularity,...,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature
0,"AdictoTainy, Anuel AA & Ozuna","Tainy, Anuel AA & Ozuna",Adicto,['pop reggaeton'],3jbT1Y5MoPwEIpZndDDwVq,NaN,Adicto (with Anuel AA & Ozuna),0.0,270740.0,91.0,...,10.0,-4.803,0.0,0.0735,0.01700,0.000016,0.1790,0.623,80.002,4.0
1,The Ones That Didn't Make It Back HomeJustin M...,Justin Moore,The Ones That Didn't Make It Back Home,"['arkansas country', 'contemporary country', '...",NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,ShallowLady Gaga & Bradley Cooper,Lady Gaga & Bradley Cooper,Shallow,"['dance pop', 'pop']",2VxeLyX666F8uXCJ0dZF8B,NaN,A Star Is Born Soundtrack,0.0,215733.0,88.0,...,7.0,-6.362,1.0,0.0308,0.37100,0.000000,0.2310,0.323,95.799,4.0
3,EnemiesPost Malone Featuring DaBaby,Post Malone Featuring DaBaby,Enemies,"['dfw rap', 'melodic rap', 'rap']",0Xek5rqai2jcOWCYWJfVCF,NaN,Hollywood's Bleeding,1.0,196760.0,86.0,...,6.0,-4.169,1.0,0.2100,0.05880,0.000000,0.0955,0.667,76.388,4.0
4,"Bacc At It AgainYella Beezy, Gucci Mane & Quavo","Yella Beezy, Gucci Mane & Quavo",Bacc At It Again,"['dfw rap', 'rap', 'southern hip hop', 'trap']",2biNa12dMbHJrHVFRt8JyO,https://p.scdn.co/mp3-preview/fa6fa6f6f363be29...,Bacc At It Again,1.0,228185.0,61.0,...,8.0,-5.725,0.0,0.1680,0.00124,0.000001,0.0716,0.856,135.979,4.0


## 1) Limpeza e Manipulação dos dados:

### [1.1] Limpeza:

Vamos começar tirando as colunas que não interferem na classificação e depois excluir as linhas que não possuem informação.

In [6]:
#=================================================== Colunas desejadas ========================================================
# escolhemos as colunas que importam para a classificação
DataFrame = DataFrame.loc[:,['spotify_genre','loudness','spotify_track_popularity','spotify_track_duration_ms',
                             'spotify_track_explicit','danceability','mode','speechiness','energy','instrumentalness',
                             'acousticness','liveness','valence']]

#==================================================== Excluindo linhas ========================================================
# a função exclui as linhas que possuem células sem informação (NaN)
DataFrame = DataFrame.dropna()

#DataFrame.head()  #debug

### [1.2] Categorizando:

As colunas `spotify_track_explicit` e `mode` são categóricas, então vamos mudar o seu tipo e escrever corretamente a caracteristica da categoria:

In [9]:
#===================================================== Categorizando ==========================================================
# Transformando a coluna em uma categoria
DataFrame['spotify_track_explicit'] = DataFrame['spotify_track_explicit'].astype('category')
DataFrame['mode'] = DataFrame['mode'].astype('category')

# Trocando os valores pelas categorias correspondentes:
DataFrame['spotify_track_explicit'].cat.categories = ['Free','Explicit']
DataFrame['mode'].cat.categories = ['Minor','Major']

#DataFrame.head()  #debug

### [1.3] Treinamento e Teste:

Agora, vamos separar o DataFrame já limpo em `Treinamento`, utilizado para treinar nosso classificador, e `Teste`, para analisar o funcionamento do classificador ja criado e treinado, sendo 70% para o Treinamento e 30% para o Teste.

In [18]:
#================================================= Criando os DataFrames ======================================================
# para o X do Dataframe novo
Lista_Colunas = list(DataFrame.columns)
Lista_Colunas.remove('spotify_genre')
X = DataFrame[Lista_Colunas]

# para o y do Dataframe novo
y = DataFrame['spotify_genre']

# aplicando a função: Treinamento com 70% do DataSet:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)